How to use a model to call tools

Passing tools to chat models

在 Python 函数上使用 @tool 装饰器来定义自定义工具的架构

In [1]:
from langchain_core.tools import tool

@tool             #装饰器（decorator），定义函数 add 的访问级别和用途
def add(a: int, b: int) -> int:       #-> int 是一个类型提示，表明函数 add 将返回一个整数结果。
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

使用 Pydantic 定义模式

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

tools = [Add, Multiply]


绑定到chat models

In [3]:
# pip install -qU langchain-openai

In [4]:
import getpass
import os

os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"


openai_api_key = os.environ["OPENAI_API_KEY"]
print(openai_api_key)

os.environ["OPENAI_API_BASE"]="https://api.zhizengzeng.com/v1/"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

sk-zk224a777126590fa5988d720e1c413bd6866854f68d0768


In [5]:
# 
llm_with_tools = llm.bind_tools(tools)
query = "What is 3 * 12?"
result = llm_with_tools.invoke(query)

print(result)


content='' additional_kwargs={'tool_calls': [{'id': 'call_OeSeaaubxhLsKL58GFp1t3JI', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 95, 'total_tokens': 113}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-c6332cec-e61c-48db-82f3-47f3da5f9bee-0' tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_OeSeaaubxhLsKL58GFp1t3JI'}] usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113}


Tool calls

In [6]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_GHayO5V3rDb3YaiiENserBwg'},
 {'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_wI7VyCGGZJVqxeZ80wVKj23n'}]

使用 PydanticToolsParser 将 .tool_calls 属性上填充的现有值转换回原始 Pydantic 类

In [7]:
from langchain_core.output_parsers import  PydanticToolsParser 

chain = llm_with_tools | PydanticToolsParser (tools = [Add, Multiply])
chain.invoke(query)

[Multiply(a=3, b=12), Add(a=11, b=49)]

How to use few-shot prompting with tool calling

In [8]:
llm_with_tools = llm.bind_tools(tools)

In [9]:
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_UgF8aF3GJnsh8WDfoqyOJdet'},
 {'name': 'Add',
  'args': {'a': 952, 'b': -20},
  'id': 'call_kjhyIv1xf2OnBndYkXrfgWJL'}]

adding a prompt with some examples

In [10]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [11]:
examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name = "example_assistant",
        tool_calls = [
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),

    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

In [12]:
system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""

few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system),
        *examples,           #接收examples并作为单独参数传递给from_messages
        ("human","{query}"),
    ]
)


In [13]:
chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_lclTCe0pGyCxkYu75l2DTapR'}]

How to stream tool calls

In [14]:
###定义查询并流式输出

query = "What is 3 * 12? Also, what is 11 + 49?"

#asnyc异步迭代器
async for chunk in llm_with_tools.astream(query):       #循环遍历异步可迭代对象并打印chunk工具调用信息
    print(chunk.tool_call_chunks)                       

[]
[{'name': 'Multiply', 'args': '', 'id': 'call_lZpDAJNeQxEfFPekSlWj88UG', 'index': 0}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0}]
[{'name': 'Add', 'args': '', 'id': 'call_zROkQdDphrv4t41Scuoyqchk', 'index': 1}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1}]
[]


积累工具调用

In [15]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk
    
    print(gathered.tool_call_chunks)

[]
[{'name': 'Multiply', 'args': '', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a"', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, ', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}, {'name': 'Add', 'args': '', 'id': 'call_3T7LBhzqx6VgSNN8Ai41w9aj', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}, {'name': 'Add', 'args': '{"a"', 'id': 'call_3T7LBhzqx6VgSNN8Ai41w9aj', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_ywVlA3UkBeRwQtV6VL1lP5rJ', 'index': 0}, {'name': 'Add', 'args': '{"a": 11,', 'id': 'call_

In [16]:
print(gathered.tool_call_chunks[0]["args"])        # 获取tool_call_chunks的第一个元素的args键的值
print(type(gathered.tool_call_chunks[0]["args"]))    

{"a": 3, "b": 12}
<class 'str'>


In [17]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

# 区别在这里
    print(gathered.tool_calls)

[]
[]
[{'name': 'Multiply', 'args': {}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}]
[{'name': 'Multiply', 'args': {'a': 3}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 1}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}, {'name': 'Add', 'args': {}, 'id': 'call_Cbz1W6NAf0CVjC0H64wI4iB8'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_Cbz1W6NAf0CVjC0H64wI4iB8'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_Cbz1W6NAf0CVjC0H64wI4iB8'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_W3tF6imY85h8IjRfw1CwSp9i'}, 

In [18]:
print(gathered.tool_calls[0]["args"])
print(type(gathered.tool_calls[0]["args"]))        
###不理解输出type不同

{'a': 3, 'b': 12}
<class 'dict'>


How to bind model-specific tools

In [19]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [20]:
model_with_tools = model.bind(
    tools=[{
        "type":"function",
        "function":{
            "name":"multiply",
            "description": "Multiply two integers together.",
            "parameters":{
                "type": "object",
                "properties":{
                    "a": {"type": "number", "description": "First integer"},
                    "b": {"type": "number", "description": "Second integer"},
                },
                "required":["a","b"],
            },
        },
    }
    ]
)

In [21]:
result = model_with_tools.invoke("Whats 119 times 8?")
print(result )

content='' additional_kwargs={'tool_calls': [{'id': 'call_bk0SzQfdns42nsiAsOZWzW1l', 'function': {'arguments': '{"a": 119, "b": 8}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 62, 'total_tokens': 94}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-66ba059f-98e1-4e10-b2d5-e555a6de38a6-0' tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_bk0SzQfdns42nsiAsOZWzW1l'}] usage_metadata={'input_tokens': 62, 'output_tokens': 32, 'total_tokens': 94}


AIMessage(content='', 
additional_kwargs={'tool_calls': [{'id': 'call_mn4ELw1NbuE0DFYhIeK0GrPe', 'function': {'arguments': '{"a":119,"b":8}', 'name': 'multiply'}, 'type': 'function'}]}, 
response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 62, 'total_tokens': 79}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, 
id='run-353e8a9a-7125-4f94-8c68-4f3da4c21120-0', 
tool_calls=[{'name': 'multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_mn4ELw1NbuE0DFYhIeK0GrPe'}])

How to pass run time values to a tool

Passing request time information
传递请求时间信息
在请求时动态创建工具，并将适当的信息绑定到它。例如，该信息可以是从请求本身解析出的用户ID。

In [22]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.tools import BaseTool , tool

In [23]:
user_to_pets = {}

def generate_tools_for_user (user_id:str)-> List[BaseTool]:
    """Generate a set of tools that have a user id associated with them."""

    @tool
    def update_favorite_pets(pets: List[str]) -> None:          # “-> None” 指定函数返回行为，表示该函数没有返回任何值
        """Add the list of favorite pets."""
        user_to_pets[user_id] = pets                            # 将用户 ID 对应的宠物列表存入 user_to_pets 字典中

    @tool
    def delete_favorite_pets() -> None:
        """Delete the list of favorite pets."""
        if user_id in user_to_pets:
            del user_to_pets[user_id]                           # 如果用户 ID 存在于 user_to_pets 中，删除对应的条目
    
    @tool
    def list_favorite_pets() -> None:
        """List favorite pets if any."""
        return user_to_pets.get(user_id, [])                    # 返回用户 ID 对应的宠物列表，如果不存在则返回空列表
    
    return [update_favorite_pets, delete_favorite_pets, list_favorite_pets]
    

In [24]:
update_pets, delete_pets, list_pets = generate_tools_for_user("eugene")   # user_id："eugene"
update_pets.invoke({"pets": ["cat", "dog"]})                              # "pets": ["cat", "dog" 
print(user_to_pets)
print(list_pets.invoke({}))

{'eugene': ['cat', 'dog']}
['cat', 'dog']


In [25]:
from langchain_core.prompts import  ChatPromptTemplate

In [30]:
def handle_run_time_request(user_id: str, query: str):
    """Handle run time request."""
    tools =  generate_tools_for_user(user_id)  
    llm_with_tools = llm.bind_tools(tools)
    prompt = ChatPromptTemplate.from_messages(
        [("system","You are a helpful assistant.")]
    )
    chain = prompt | llm_with_tools
    return llm_with_tools.invoke(query)


此代码将允许 LLM 调用这些工具，但 LLM 不知道用户 ID 的存在

In [31]:
ai_message = handle_run_time_request(
    "eugene", "my favorite animals are cats and parrots."
)
ai_message.tool_calls

[{'name': 'update_favorite_pets',
  'args': {'pets': ['cats', 'parrots']},
  'id': 'call_cTIQwIggrgkLW3hF7WjCdMlR'}]

How to pass tool outputs to the model

使用 ToolMessages 调用模型生成的工具并将结果传递回模型

In [ ]:
# 定义工具和函数（见上）
# 连接model（gpt-3.5-turbo-0125）

In [37]:
# 使用 ToolMessage 将工具调用的输出传递回模型
from langchain_core.messages import HumanMessage, ToolMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]
print(f"messsge:",messages)

ai_msg = llm_with_tools.invoke(messages)
print(f"ai_msg:",ai_msg)
print(ai_msg.tool_calls)

messages.append(ai_msg)
print(f"messsge_append:",messages)

messsge: [HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?')]
ai_msg: content='' additional_kwargs={'tool_calls': [{'id': 'call_qiSpaAeD5G2koM4ztbNSAjg7', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_AkSdXuyJ89FixDSPIDKehfAD', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-9e2dca10-f5fc-43ab-8115-128050234e44-0' tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qiSpaAeD5G2koM4ztbNSAjg7'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_AkSdXuyJ89FixDSPIDKehfAD'}] usage_metadata={'input_tokens': 105, 'output_tokens': 50, 'total_tokens': 155}
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qiSpaAeD5G2koM4ztbNSAjg7'}, {

messsge: [HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?')]

ai_msg: content='' additional_kwargs={'tool_calls': [{'id': 'call_Roj8n2qoYaPVgbqflMYuRjql', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_2wCh9R62WGL16H2P8DT8ll6P', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]} 
response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}
id='run-1e56810d-c107-4e60-84ca-faaace650d37-0' tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_Roj8n2qoYaPVgbqflMYuRjql'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_2wCh9R62WGL16H2P8DT8ll6P'}] usage_metadata={'input_tokens': 105, 'output_tokens': 50, 'total_tokens': 155}

messsge_append: [HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'), 
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Roj8n2qoYaPVgbqflMYuRjql', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_2wCh9R62WGL16H2P8DT8ll6P', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]},
response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, 
id='run-1e56810d-c107-4e60-84ca-faaace650d37-0', 
tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_Roj8n2qoYaPVgbqflMYuRjql'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_2wCh9R62WGL16H2P8DT8ll6P'}], usage_metadata={'input_tokens': 105, 'output_tokens': 50, 'total_tokens': 155})]


In [40]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add":add,"multiply": multiply}[tool_call["name"].lower()]
    # print(f"selected_tool:",selected_tool)

    tool_output = selected_tool.invoke(tool_call["args"])
    # print(f"tool_output:",tool_output)

    # # 使用 ToolMessage 记录工具函数执行结果
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))      

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qiSpaAeD5G2koM4ztbNSAjg7', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_AkSdXuyJ89FixDSPIDKehfAD', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9e2dca10-f5fc-43ab-8115-128050234e44-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_qiSpaAeD5G2koM4ztbNSAjg7'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_AkSdXuyJ89FixDSPIDKehfAD'}], usage_metadata={'input_tokens': 105, 'output_tokens': 50, 'total_tokens': 155}),
 ToolMessage(content='36', tool_call_id='call_qiSpaAeD5G2koM4ztbNSAjg7'),
 ToolMessage(co

In [41]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 is 36, and 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 171, 'total_tokens': 190}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ad84148-2f67-4185-bff8-6429d67e354e-0', usage_metadata={'input_tokens': 171, 'output_tokens': 19, 'total_tokens': 190})

AIMessage(content='3 * 12 is 36, and 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 171, 'total_tokens': 190}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ad84148-2f67-4185-bff8-6429d67e354e-0', usage_metadata={'input_tokens': 171, 'output_tokens': 19, 'total_tokens': 190})